In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('./dataset/clean_data.csv').head(1000)
df.head()

,is_profane,text
0,0,Then go to the village pump and suggest they c...
1,1,ANTI GREEK NATIONALIS -WIKIPEDIA \r\n\r\nHi Al...
2,1,Dis hoe wasnt dis violent on Lottery Ticket 😂😂
3,0,It is better for Atabay not helping the banned...
4,0,"""is in CamelCase. """"SiCKO"""" is not CamelCase,..."


In [3]:
sentences = df['text']
labels = df['is_profane']

max_words = 5000
max_len = 50

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, GRU, SpatialDropout1D
from sklearn.model_selection import train_test_split

In [30]:
embedding_dim = 128

model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=max_len))
model.add(SpatialDropout1D(0.2))
# model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(GRU(64, return_sequences=False))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

c:\Users\ssharma\AppData\Local\miniforge3\lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


<Sequential name=sequential_3, built=False>

In [32]:
X_valtrain, X_valtest, y_valtrain, y_valtest = train_test_split(sentences, labels, test_size=0.2, random_state=42)

tokenizer = Tokenizer(num_words=max_words, lower=True)
tokenizer.fit_on_texts(sentences)

sequences = tokenizer.texts_to_sequences(sentences)
padded_sequences = pad_sequences(sequences, maxlen=max_len)

X_train, X_test, y_train, y_test = train_test_split(padded_sequences, labels, test_size=0.2, random_state=42)

batch_size=32
epochs=5

history = model.fit(X_train, np.array(y_train), epochs=epochs, batch_size=batch_size, 
    validation_data=(X_test, np.array(y_test)), verbose=2)

print(model.summary())

loss, accuracy = model.evaluate(X_test, np.array(y_test), verbose=2)
print(f'Accuracy: {accuracy*100: .2f}% -- Loss: {loss*100:.2f}%')

Epoch 1/5
25/25 - 1s - 36ms/step - accuracy: 0.9937 - loss: 0.0327 - val_accuracy: 0.9050 - val_loss: 0.3885
Epoch 2/5
25/25 - 1s - 31ms/step - accuracy: 0.9950 - loss: 0.0235 - val_accuracy: 0.8750 - val_loss: 0.3718
Epoch 3/5
25/25 - 1s - 31ms/step - accuracy: 1.0000 - loss: 0.0109 - val_accuracy: 0.8950 - val_loss: 0.4211
Epoch 4/5
25/25 - 1s - 31ms/step - accuracy: 0.9975 - loss: 0.0107 - val_accuracy: 0.8950 - val_loss: 0.4101
Epoch 5/5
25/25 - 1s - 35ms/step - accuracy: 1.0000 - loss: 0.0045 - val_accuracy: 0.8900 - val_loss: 0.4211


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)         │ (32, 50, 128)          │       640,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout1d_3             │ (32, 50, 128)          │             0 │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_1 (GRU)                     │ (32, 64)               │        37,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (32, 1)                │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,031,941 (7.75 MB)

 Trainable params: 677,313 (2.58 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,354,628 (5.17 MB)

None
7/7 - 0s - 15ms/step - accuracy: 0.8900 - loss: 0.4211
Accuracy:  89.00% -- Loss: 42.11%


In [29]:
def predict(sentence):
    sequence = tokenizer.texts_to_sequences([sentence])
    padded_sequence = pad_sequences(sequence, maxlen = max_len)
    prediction = model.predict(padded_sequence)
    # print(prediction)
    return prediction[0][0]

test_sentences = [
    "I love you",
    "You are so dumb",
    "This is fantastic",
    "You are a fool",
    "Biatch is hoe",
    "ANTI GREEK NATIONALIS -WIKIPEDIA",
    "Dis hoe wasnt dis violent on Lottery Ticket",
    "Hi Bruv, Hope you are doing well", 
    "Love this product", 
    "She's a hoe", 
    'Suck water from bottle', 
    "write the article ibtch", 
    "Idiot, A cup of cum for you! DRINK IT ALL PLZ", 
    "A cup of cum for you! DRINK IT ALL PLZ"
]

for sentence, truth in zip(X_valtest, y_valtest):
    prediction = predict(sentence)
    print("="*30)
    print(f'Sentences: {sentence}\nTruth: {truth}\nProfanity probability: {prediction*100:.2f}%')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 307ms/step
Sentences: @zt0mm lucky monkey
Truth: 0
Profanity probability: 59.01%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
Sentences: TheManWhoLaughs

Can you believe this troll? He's the one who kept removing the secondary theme for WrestleMania 22. He's calling me a troll and say I am harassing him by putting a warning on his talk page, and says that WWE is wrong. Some people can be so annoying, especially when they think they are right. TJ Spyke 

So Im a troll eh? youre the one with false information. And upon that note Ill ask to stay the fuck away from me and my edits.
Truth: 1
Profanity probability: 23.68%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
Sentences: "

 Newest vandalisms by Tajik as anon IP 

He persianises the article Timurid dynasty with thousands of lies and with exploiting the memory of the dead of the Armnenian genocide to discredit me suggesting me as a Turk to better discredit me with the Armenian genocide and let his version ""win"" via this 